# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
from scipy import stats

# Import API key
from api_keys import geoapify_key


In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\Jeremy Morris\OneDrive\Desktop\OSU Data Analysis Course\Modules\Module 6\WeatherPy\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,lat,lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,9.25,57,0,6.17,CL,1712766464
1,1,la serena,-29.9078,-71.2542,14.73,94,100,2.06,CL,1712766453
2,2,tura,25.5198,90.2201,24.91,28,80,2.02,IN,1712766616
3,3,anadyr,64.7500,177.4833,-8.72,92,75,3.00,RU,1712766616
4,4,papatowai,-46.5619,169.4708,12.39,99,100,1.27,NZ,1712766616


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'lon',
    'lat',
    geo=True,
    color='Humidity',
    size=80,
    tiles="OSM",  
    frame_width=800,
    frame_height=600
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Filter the DataFrame based on the specified conditions
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 27) & 
                                 (city_data_df["Max Temp"] > 21) & 
                                 (city_data_df["Wind Speed"] < 4.5) & 
                                 (city_data_df["Cloudiness"] == 0)]

# Display the filtered DataFrame
ideal_weather_df 


# Drop any rows with null values
ideal_weather_conditions = ideal_weather_df.dropna()
# locations=ideal_weather_df[['lat', 'lon']].astype(float)

# Display sample data
ideal_weather_conditions

,City_ID,City,lat,lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,tolanaro,-25.0319,46.9987,25.04,88,0,2.06,MG,1712766618
89,89,lompoc,34.6391,-120.4579,21.55,72,0,2.06,US,1712766627
168,168,rudbar,30.1500,62.6000,26.87,18,0,1.92,AF,1712766635
207,207,paradise,36.0972,-115.1467,21.63,23,0,2.57,US,1712766638
253,253,grenada,37.1882,-3.6067,23.84,32,0,4.12,ES,1712766642
262,262,cerritos,33.8583,-118.0648,24.37,52,0,2.06,US,1712766633
272,272,pisco,-13.7000,-76.2167,25.57,78,0,4.12,PE,1712766644
299,299,armasesti,44.7500,26.5833,22.42,34,0,3.07,RO,1712766646
338,338,la palma,33.8464,-118.0467,24.39,49,0,2.06,US,1712766650
344,344,awjilah,29.1081,21.2869,23.08,21,0,3.99,LY,1712766651


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_conditions[["City", "Country", "lat", "lon", "Humidity" ]].copy()



# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,lat,lon,Humidity,Hotel Name
19,tolanaro,MG,-25.0319,46.9987,88,
89,lompoc,US,34.6391,-120.4579,72,
168,rudbar,AF,30.1500,62.6000,18,
207,paradise,US,36.0972,-115.1467,23,
253,grenada,ES,37.1882,-3.6067,32,
262,cerritos,US,33.8583,-118.0648,52,
272,pisco,PE,-13.7000,-76.2167,78,
299,armasesti,RO,44.7500,26.5833,34,
338,la palma,US,33.8464,-118.0467,49,
344,awjilah,LY,29.1081,21.2869,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
limit=1
radius = 10000
# filters=f"circle:{radius}"
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": limit
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["lon"]
    latitude = row["lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tolanaro - nearest hotel: Hôtel Mahavokey
lompoc - nearest hotel: Red Roof Inn Lompoc
rudbar - nearest hotel: No hotel found
paradise - nearest hotel: Best Western McCarran Inn
grenada - nearest hotel: Allegro Granada
cerritos - nearest hotel: Sheraton
pisco - nearest hotel: La Portada
armasesti - nearest hotel: Pensiunea Apollo
la palma - nearest hotel: La Quinta Inn & Suites by Wyndham Buena Park
awjilah - nearest hotel: No hotel found
ciudad gustavo diaz ordaz - nearest hotel: No hotel found
cudahy - nearest hotel: No hotel found


,City,Country,lat,lon,Humidity,Hotel Name
19,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
89,lompoc,US,34.6391,-120.4579,72,Red Roof Inn Lompoc
168,rudbar,AF,30.1500,62.6000,18,No hotel found
207,paradise,US,36.0972,-115.1467,23,Best Western McCarran Inn
253,grenada,ES,37.1882,-3.6067,32,Allegro Granada
262,cerritos,US,33.8583,-118.0648,52,Sheraton
272,pisco,PE,-13.7000,-76.2167,78,La Portada
299,armasesti,RO,44.7500,26.5833,34,Pensiunea Apollo
338,la palma,US,33.8464,-118.0467,49,La Quinta Inn & Suites by Wyndham Buena Park
344,awjilah,LY,29.1081,21.2869,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
# %%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'lon',
    'lat',
    geo=True,
    color='Hotel Name',
    size=80,
    tiles="OSM",  
    frame_width=800,
    frame_height=600
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [lon,lat]   (Hotel Name)